# BDA1 - Spark

In this set of exercises you will work exclusively with Spark. This means that in your programs, you only need to create the `SparkContext`.
In a number of exercises you will be asked to calculated temperature averages (daily and monthly). These are not always computed according to the standard definition of ‘average’. In this domain the daily average temperature is calculated by averaging the daily measured maximum and the daily measured minimum temperatures. The monthly average is calculated by averaging the daily maximums and minimums for that month. For example, to get the monthly average for October, take maximums and minimums for each day, sum them up and divide by 62 (which is the same as taking the daily averages, summing them up and divide by the number of days).

Note: In many countries in the world, the averages are calculated as discussed. However, in Sweden, daily and monthly averages are calculated using Ekholm-Modéns formula which in addition to minimum and maximum daily temperature also takes into account readings at specific timepoints, the month as well as the longitude of the station. For more information check (in Swedish): http://www.smhi.se/kunskapsbanken/meteorologi/hur-beraknas-medeltemperatur-1.3923

## Setup

The following code is used to prepare the data files, execute only once and **be careful!**

In [1]:
import pyspark
import csv
import math
import sys
import pandas as pd

tr_path = "../data/temperature-readings.csv"
tr_big_path = "../data/temperature-readings-big.csv"
pr_path = "../data/precipitation-readings.csv"
sro_path = "../data/stations-Ostergotland.csv"
sr_path = "../data/stations.csv"

In [2]:
sc = pyspark.SparkContext(appName = "Temperature")
temperature_readings = sc.textFile(tr_path)
temperature_readings_big = sc.textFile(tr_big_path)
precipitation_readings = sc.textFile(pr_path)
station_oster_readings = sc.textFile(sro_path)
station_readings = sc.textFile(sr_path)

## Assignment 1

What are the lowest and highest temperatures measured each year for the period 1950- 2014. Provide the lists sorted in the descending order with respect to the maximum temperature. In this exercise you will use the `temperature-readings.csv` file.

In [3]:
%%time

global res

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda l: (int(l[1][0:4]), float(l[3])))
res = res.filter(lambda m: m[0] >= 1950 and m[0] <= 2014)

res_max = res.reduceByKey(max)
res_max = res_max.map(lambda m: (m[1], m[0]))
res_min = res.reduceByKey(min)
res_min = res_min.map(lambda m: (m[1], m[0]))

res = res_max.zip(res_min).map(lambda m: (m[0][1], m[0][0], m[1][0]))
res = res.sortBy(lambda m: m[0], ascending=False)

global df
df = pd.DataFrame(res.collect(), columns = ['year', 'temp_max', 'temp_min'])

CPU times: user 84.5 ms, sys: 38 ms, total: 123 ms
Wall time: 3min 11s


In [4]:
df.head(10)

,year,temp_max,temp_min
0,2014,34.4,-42.5
1,2013,31.6,-40.7
2,2012,31.3,-42.7
3,2011,32.5,-42.0
4,2010,34.4,-41.7
5,2009,31.5,-38.5
6,2008,32.2,-39.3
7,2007,32.2,-40.7
8,2006,32.7,-40.6
9,2005,32.1,-39.4


**Task 1a:** What are the lowest and highest temperature measured each year for the period 1950 to 2014? Provide the list sorted in the descending order with respect to the maximum temperature. Extend the program to include the station number (not the station name) where the maximum/minimum temperature was measured.

In [5]:
%%time

global res

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda l: (int(l[1][0:4]), (int(l[0]), float(l[3]))))
res = res.filter(lambda m: m[0] >= 1950 and m[0] <= 2014)

res_max = res.reduceByKey(lambda x, y: max(x, y, key=lambda z: float(z[1])))
res_max = res_max.map(lambda m: (m[0], m[1][1], m[1][0]))
res_min = res.reduceByKey(lambda x, y: min(x, y, key=lambda z: float(z[1])))
res_min = res_min.map(lambda m: (m[1][1], m[1][0]))

res = res_max.zip(res_min).map(lambda m: (m[0][0], m[0][1], m[0][2], m[1][0], m[1][1]))
res = res.sortBy(lambda m: m[0], ascending = False)

global df
df = pd.DataFrame(res.collect(), columns = ['year', 'temp_max', 'station_max', 'temp_min', 'station_min'])

CPU times: user 96.5 ms, sys: 18.9 ms, total: 115 ms
Wall time: 3min 58s


In [6]:
df.head()

,year,temp_max,station_max,temp_min,station_min
0,2014,34.4,96560,-42.5,192840
1,2013,31.6,98210,-40.7,179960
2,2012,31.3,54990,-42.7,191910
3,2011,32.5,172770,-42.0,179960
4,2010,34.4,75250,-41.7,191910


**Task 1b**: Write the non-parallelized program in Python to find the maximum temperatures for each year without using Spark.

In [7]:
def csv_read_lazy(csvfile, delimiter = ",", encoding = "utf-8"):
    with open(csvfile, encoding = encoding) as f:
        r = csv.reader(f, delimiter = delimiter)
        for row in r:
            yield row

In [8]:
%%time

def get_temperature_information(min_year = 1950, max_year = 2014):
    
    temp_info = {}
    temp_info["max"] = {}
    temp_info["min"] = {}
    
    for year in range(min_year, max_year + 1):
        temp_info["max"][year] = {"temp": float("-Inf"), "station": None}
        temp_info["min"][year] = {"temp": float("Inf"), "station": None}

    for row in csv_read_lazy(tr_path, delimiter = ";"):
        year = int(row[1][0:4])
        station = int(row[0])
        temp = float(row[3])
    
        if year >= min_year and year <= max_year:
            # Max
            if temp > temp_info["max"][year]["temp"]:
                temp_info["max"][year]["temp"] = temp
                temp_info["max"][year]["station"] = station
            
            # Min
            if temp < temp_info["min"][year]["temp"]:
                temp_info["min"][year]["temp"] = temp
                temp_info["min"][year]["station"] = station
                
    df = pd.DataFrame(columns=['year', 'temp_max', 'station_max', 'temp_min', 'station_min'])
        
    for year, v in temp_info["max"].items():
        if v["temp"] != float("-Inf"):
            df.loc[len(df)] = [year, v["temp"], v["station"], temp_info["min"][year]["temp"], temp_info["min"][year]["station"]]
    
    df["year"] = df["year"].astype(int)
    df["station_max"] = df["station_max"].astype(int)
    df["station_min"] = df["station_min"].astype(int)
    
    return df.sort_values(by = ['temp_max'], ascending = False)

global df
df = get_temperature_information()

CPU times: user 2min 3s, sys: 755 ms, total: 2min 4s
Wall time: 2min 4s


In [9]:
df.head(10)

,year,temp_max,station_max,temp_min,station_min
25,1975,36.1,86200,-37.0,157860
42,1992,35.4,63600,-36.1,179960
44,1994,34.7,117160,-40.5,179960
64,2014,34.4,96560,-42.5,192840
60,2010,34.4,75250,-41.7,191910
39,1989,33.9,63050,-38.2,166870
32,1982,33.8,94050,-42.2,113410
18,1968,33.7,137100,-42.0,179950
16,1966,33.5,151640,-49.4,179950
33,1983,33.3,98210,-38.2,191900


As we can see the non-parallized version is actually more than twice as fast than the parallized version. Apparently PySpark adds a lot of overhead. I ran this on my PC with a i7-2600k, which has 4 cores and can execute 8 threads using hyper threading, so even if it's *just* a factor of 4/8, the parallized version should be faster the the non-parallized one, so the implementation seems not to be flawless.

## Assignment 2

**Task 2:** Count the number of readings for each month in the period of 1950-2014 which are higher than 10 degrees. Repeat the exercise, this time taking only distinct readings from each station. That is, if a station reported a reading above 10 degrees in some month, then it appears only once in the count for that month. In this exercise you will use the `temperature-readings.csv` file.

The output should contain the following information:

Year, month, count

In [10]:
%%time
res = temperature_readings.map(lambda l: l.split(";"))

res = res.map(lambda l: (int(l[1][0:4]), int(l[1][5:7]), float(l[3])))
res = res.filter(lambda m: int(m[0]) >= 1950 and int(m[0]) <= 2014 and m[2] > 10.0)
res = res.map(lambda m: (str(m[0]) + "-" + str(m[1]).zfill(2), 1))
res = res.reduceByKey(lambda x, y: x + y)
res = res.map(lambda m: (m[0][0:4], m[0][5:7], m[1]))
res = res.sortBy(lambda m: (m[0], m[1]), ascending = False)

global df
df = pd.DataFrame(res.collect(), columns = ['year', 'month', 'count'])

CPU times: user 71.4 ms, sys: 20.1 ms, total: 91.6 ms
Wall time: 1min 32s


In [11]:
df.head(10)

,year,month,count
0,2014,12,3
1,2014,11,8139
2,2014,10,42191
3,2014,09,86090
4,2014,08,124045
5,2014,07,147681
6,2014,06,101711
7,2014,05,57250
8,2014,04,19862
9,2014,03,4213


Now we repeat the exercise where we take only distinct readings from each station.

In [12]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda l: (int(l[0]), int(l[1][0:4]), int(l[1][5:7]), float(l[3])))
res = res.filter(lambda m: int(m[1]) >= 1950 and int(m[1]) <= 2014 and m[3] > 10.0)
res = res.map(lambda m: (str(m[0]) + "-" + str(m[1]) + "-" + str(m[2]).zfill(2), 1))
res = res.reduceByKey(lambda x, y: x + y)
res = res.map(lambda m: (m[0][-7:], 1))
res = res.reduceByKey(lambda x, y: x + y)
res = res.sortBy(lambda m: m[0], ascending=False)
res = res.map(lambda m: (m[0][0:4], m[0][5:7], m[1]))

global df
df = pd.DataFrame(res.collect(), columns = ['year', 'month', 'count'])

CPU times: user 85.6 ms, sys: 16.1 ms, total: 102 ms
Wall time: 1min 32s


In [13]:
df.head(10)

,year,month,count
0,2014,12,1
1,2014,11,158
2,2014,10,270
3,2014,09,296
4,2014,08,296
5,2014,07,297
6,2014,06,298
7,2014,05,296
8,2014,04,254
9,2014,03,169


## Assignment 3

Find the average monthly temperature for each available station in Sweden. Your result should include average temperature for each station for each month in the period of 1960-2014. Bear in mind that not every station has the readings for each month in this timeframe. In this exercise you will use the `temperature-readings.csvfile`.

The output should contain the following information:

Year, month, station number, average monthly temperature

In [14]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda m: (m[0] + "-" + m[1][0:7], int(m[1][0:4]), float(m[3])))
res = res.filter(lambda m: m[1] >= 1960 and m[1] <= 2014)
res = res.map(lambda m: (m[0], (m[2], 1)))
res = res.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
res = res.map(lambda m: (int(m[0][:-8]), int(m[0][-7:-3]), str(m[0][-2:]).zfill(2), round(m[1][0]/m[1][1], 2)))
res = res.sortBy(lambda m: (m[1], m[2], m[0]), ascending=False)

global df
df = pd.DataFrame(res.collect(), columns=['station', 'year', 'month', 'average-temperature'])

CPU times: user 271 ms, sys: 40 ms, total: 311 ms
Wall time: 1min 42s


In [15]:
df.head(10)

,station,year,month,average-temperature
0,192840,2014,12,-11.48
1,191910,2014,12,-10.86
2,191720,2014,12,-11.33
3,189720,2014,12,-9.84
4,188850,2014,12,-7.64
5,188820,2014,12,-7.58
6,188800,2014,12,-6.94
7,183750,2014,12,-8.90
8,182930,2014,12,-9.77
9,182910,2014,12,-9.38


## Assignment 4

Provide  a  list  of  stations  with  their  associated  maximum  measured  temperatures  and maximum  measured  daily  precipitation.  Show  only  those  stations  where  the  maximum temperature is between 25 and 30 degrees and maximum daily precipitation is between 100 mm and 200mm. In this exercise you will use the `temperature-readings.csv` and `precipitation-readings.csv` files. 

The output should contain the following information:

Station number, maximum measured temperature, maximum daily precipitation

In [16]:
%%time

res_p = precipitation_readings.map(lambda l: l.split(";"))
res_t = temperature_readings.map(lambda l: l.split(";"))

res_p = res_p.map(lambda m: (int(m[0]), float(m[3])))
res_p = res_p.filter(lambda m: m[1] > 100 and m[1] < 200)
res_p = res_p.reduceByKey(lambda x, y: max(x, y))

res_t = res_t.map(lambda m: (int(m[0]), float(m[3])))
res_t = res_t.filter(lambda m: m[1] > 20 and m[1] < 30)
res_t = res_t.reduceByKey(lambda x, y: max(x, y))

res = res_t.join(res_p)
res = res.map(lambda m: (m[0], m[1][0], m[1][1]))

global df
df = pd.DataFrame(res.collect(), columns=['station', 'max_temperature', 'max_precipitation'])

CPU times: user 64.8 ms, sys: 16 ms, total: 80.8 ms
Wall time: 1min 30s


In [17]:
df.head()

,station,max_temperature,max_precipitation


## Assignment 5

Calculate the average monthly precipitation for the Östergotland region (list of stations is provided in the separate file) for the period 1993-2016. In order to do this, you will first need to calculate  the  total  monthly  precipitation  for  each  station  before calculating the monthly average (by averaging over stations). In this exercise you will use the `precipitation-readings.csv` and `stations-Ostergotland.csv` files.

HINT (not for the SparkSQL lab): Avoid using joins here! `stations-Ostergotland.csv` is small  and if distributed will cause a number of unnecessary shuffles when joined with precipitation RDD. If you distribute `precipitation-readings.csv` then either repartition your stations RDD to 1 partition or make use of the collect to acquire a python list and broadcast function to broadcast the list to all nodes.

The output should contain the following information:

Year, month, average monthly precipitation

In [18]:
%%time

# Don't remove those, it will break as we're using %%time
global res_p
global res_s

res_p = precipitation_readings.map(lambda l: l.split(";"))
res_s = station_oster_readings.map(lambda l: l.split(";"))

res_s = res_s.map(lambda m: int(m[0])).collect()
res_s = sc.broadcast(value=res_s)

res_p = res_p.map(lambda m: (int(m[0]), m[1], float(m[3])))
res_p = res_p.filter(lambda m: int(m[1][0:4]) >= 1993 and int(m[1][0:4]) <= 2016)

res = res_p.filter(lambda m: m[0] in res_s.value)
# This changes the amount of entries as it only takes year-month, not year-month-date, thus the resulting number of measurementes will me lower. Which one is correct?
#res = res.map(lambda m: (str(m[0]) + "-" + m[1][0:7], m[2]))
res = res.map(lambda m: (str(m[0]) + "-" + m[1][0:10], m[2]))
res = res.reduceByKey(lambda x, y: x + y)

res = res.map(lambda m: (m[0][6:10] + "-" + m[0][11:13], (m[1], 1)))
res = res.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
res = res.map(lambda m: (int(m[0][0:4]), m[0][5:8], m[1][0]/m[1][1]))
res = res.sortBy(lambda m: (m[0], m[1]), ascending=False)

global df
df = pd.DataFrame(res.collect(), columns=['year', 'month', 'average_precipitation'])

CPU times: user 79.3 ms, sys: 709 µs, total: 80 ms
Wall time: 33.1 s


In [19]:
df.head(10)

,year,month,average_precipitation
0,2016,07,0.000000
1,2016,06,1.588750
2,2016,05,0.943548
3,2016,04,0.896667
4,2016,03,0.643952
5,2016,02,0.743534
6,2016,01,0.723077
7,2015,12,0.936842
8,2015,11,2.129583
9,2015,10,0.072984


## Assignment 6

Compare the average monthly temperature (find the difference) in the period 1950-2014 for all stations in Östergotland with long-term monthly averages in the period of 1950-1980. Make a plot of your results. 

HINT: The first step is to find the monthly averages for each station. Then, you can average over all stations to acquire the average temperature for a specific year and month. This RDD/DataFrame can beu sed to compute the long-term average by averaging over all the years in the interval.

The out put should contain the following information:

Year, month, difference

**Answer:** It seems like this exercise has been removed from last year to this year, as the instructions have chnaged from 1. april to 2. april 2019.

In [20]:
sc.stop()